In [1]:
from benchmark_consts import get_problems, get_args_and_problems, print_, PATH_FORM_HYPERPARAMS


import os
import pickle
import traceback
import argparse
import random
import math
import copy

import sys
sys.path.append('..')

from lib.algorithms import PathFormulation
from lib.problem import Problem
from lib.algorithms.abstract_formulation import Objective
from lib.graph_utils import compute_in_or_out_flow, path_to_edge_list, assert_flow_conservation, check_feasibility
from collections import defaultdict
import ncflow

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
startTime = datetime.now()

TOP_DIR = 'path-form-logs'
HEADERS = [
    'problem', 'num_nodes', 'num_edges', 'traffic_seed', 'scale_factor',
    'tm_model', 'num_commodities', 'total_demand', 'algo', 'num_paths',
    'edge_disjoint', 'dist_metric', 'total_flow', 'runtime'
]
PLACEHOLDER = ','.join('{}' for _ in HEADERS)


In [32]:
# subproblem_list: list of lists, one for each subproblem, containing assigned entities
# input_set_dims: dictionary mapping entity to its dimensions
# calculate mean value of every dimension of each subproblem and return
def check_dims(subproblem_list, input_set_dims):
    
    num_subproblems = len(subproblem_list)
    num_dimensions = len(list(input_set_dims.values())[0])
    print("checking split of " + str(num_dimensions) + " dimensions over " + 
          str(num_subproblems) + " subproblems")
    
    subproblem_dim_sums = [np.zeros(num_dimensions) for _ in range(num_subproblems)]
    
    for k in range(num_subproblems):
        subproblem_entities = subproblem_list[k]
        
        for entity in subproblem_entities:
            entity_dims = input_set_dims[entity]
            subproblem_dim_sums[k] = np.add(np.asarray(entity_dims), subproblem_dim_sums[k])
        subproblem_dim_sums[k] = subproblem_dim_sums[k]/len(subproblem_entities)
        print("subproblem " + str(k) + ": " + str(subproblem_dim_sums[k]))
    return subproblem_dim_sums

# estimate of covariance with new entity included
def calc_cov_online(d1, d2, current_cov, num_entity, mean_d1, mean_d2):

    new_mean_d1 = (mean_d1*num_entity + d1)/(num_entity+1) 
    new_mean_d2 = (mean_d2*num_entity + d2)/(num_entity+1) 
    
    new_cov = current_cov*(num_entity-1) + \
              (num_entity/(num_entity-1))*(d1 - new_mean_d1)*(d2 - new_mean_d2)
    new_cov = new_cov/num_entity
    return new_cov

# calculate the change in MSE between subproblem inputs and all inputs covariance

def calc_dist_cov_change(input_set_dims, new_entity, origin_dist_covs, 
                         num_entity_mean, current_covs):
    num_dims = len(new_entity)
    #print("old covs: " + str(current_covs))

    new_covs = np.zeros((num_dims,num_dims))
    if num_entity_mean[0][0] > 0:
        # calculate it from scratch for the first 50 elements
        if (num_entity_mean[0][0] < 50):
            input_set_dims_new = copy.deepcopy(input_set_dims)
            for d in range(num_dims):
                input_set_dims_new[d] += [new_entity[d]]
            new_covs = np.cov(input_set_dims_new)
        # use online update estimate
        else:
            for d1 in range(num_dims):
                # covariance matrix is symmetric, so compute only upper triangle
                for d2 in range(d1, num_dims):
                    updated_cov = calc_cov_online(new_entity[d1], new_entity[d2], 
                                                  current_covs[d1,d2], num_entity_mean[d1][0],
                                                  num_entity_mean[d1][1], num_entity_mean[d2][1])
                    new_covs[d1,d2] = updated_cov
                    new_covs[d2,d1] = updated_cov
    
    #print("new cov: " + str(new_covs))
    
    old_mse = ((current_covs - origin_dist_covs)**2).mean(axis=None)
    new_mse = ((new_covs - origin_dist_covs)**2).mean(axis=None)
    
    dist_diff = old_mse - new_mse
    return dist_diff, new_covs

# Compute the change in distance (2-norm of dimensional means)
# from the original problem inputs when adding new entity
# TODO: better distance metric that considers covariance?
def calc_dist_mean_change(input_set_dims, new_entity, origin_dist, num_entity_mean):
    num_dims = len(new_entity)
    
    sq_sum_distance = 0
    sq_sum_distance_new = 0
    for d in range(num_dims):
        if len(input_set_dims[d]) == 0:
            current_mean = 0
        else:
            #current_mean = np.mean(input_set_dims[d])
            current_mean = num_entity_mean[d][1]
            
        #new_mean = np.mean(input_set_dims[d]+[new_entity[d]])
        new_mean = (num_entity_mean[d][1]*num_entity_mean[d][0] + new_entity[d]) \
                    /(num_entity_mean[d][0]+1)
        sq_sum_distance_new += ((new_mean - origin_dist[d])/origin_dist[d])**2
        sq_sum_distance += ((current_mean - origin_dist[d])/origin_dist[d])**2
    
    return math.sqrt(sq_sum_distance) - math.sqrt(sq_sum_distance_new)
        

# input_dict: keys are entities and values are (ordered) list of entity dimensions, 
# k: number of subproblems
def split_generic(input_dict, k, verbose=False):
    
    num_inputs = len(input_dict)
    num_dimensions = len(list(input_dict.values())[0])
    
    # original_dist_dict: keys are dimension indices (0..d), values are frequency
    original_dist_means_dict = {}
    original_dist_inputs_by_dim = []
    for d in range(num_dimensions):
        inputs = [val[d] for val in input_dict.values()]
        original_dist_inputs_by_dim.append(inputs)
        sum_d = sum(inputs)
        original_dist_means_dict[d] = sum_d/(num_inputs*1.0)
    
    original_dist_cov = np.cov(original_dist_inputs_by_dim)
    
    # subproblem_dim_lists has k lists, one for each subproblem. Within each list are d lists,
    # each containing the value of a dimension for each entity assigned to that subproblem
    subproblem_dim_lists =  [[[] for _ in range(num_dimensions)] for _ in range(k)]
    
    # subproblem_entity_assignments is a list of lists
    subproblem_entity_assignments = [[] for _ in range(k)]
    
    # Assign each entity to the sub-problem that would have their distance from the
    # original distribution shrink the most.
    num_assigned = 0
    subproblem_num_entity_means = [[[0,0] for _ in range(num_dimensions)] for _ in range(k)]
    subproblem_covs = [np.zeros((num_dimensions,num_dimensions)) for _ in range(k)]
    for entity, dims in input_dict.items():
        if num_assigned % 1000 == 0:
            print("Assigned " + str(num_assigned) + " entities")
        max_dist_change = -np.inf
        max_dist_sp = 0
        updated_cov = None
        for i in range(k):
            # skip those that have more than equal share of currently assigned entities
            if len(subproblem_entity_assignments[i]) > num_assigned/(k*1.0):
                continue
                
            #dist_change = calc_dist_mean_change(subproblem_dim_lists[i], dims, 
            #                               original_dist_means_dict, subproblem_num_entity_means[i])
            dist_change, new_cov = calc_dist_cov_change(subproblem_dim_lists[i], dims, 
                                           original_dist_cov, subproblem_num_entity_means[i],
                                                       subproblem_covs[i])
            #print("subproblem " + str(i) + ", dist change: " + str(dist_change))
            if dist_change >= max_dist_change:
                max_dist_change = dist_change
                max_dist_sp = i
                #updated_cov = new_cov
                
        subproblem_entity_assignments[max_dist_sp].append(entity)
        #subproblem_covs[max_dist_sp] = new_cov
        
        # update means to reflect entity assignment
        for d in range(num_dimensions):
            subproblem_dim_lists[max_dist_sp][d].append(dims[d])
            
            num_entity = subproblem_num_entity_means[max_dist_sp][d][0]
            dim_mean = subproblem_num_entity_means[max_dist_sp][d][0]
            subproblem_num_entity_means[max_dist_sp][d][0] += 1
            subproblem_num_entity_means[max_dist_sp][d][1] = (dim_mean*num_entity + dims[d])/(num_entity+1)
        
        num_assigned += 1
        
        if verbose:
            print(subproblem_dim_lists)
            print(subproblem_entity_assignments)
            print('\n')
    return subproblem_entity_assignments

In [30]:
original_dist = [2,5]
A = [[1,2,3], [4,5,6]]
B = [2,5]
C = [3,6]
#print(calc_dist(A,B,original_dist))
#print(calc_dist(A,C,original_dist))

input_dict = {'A': [1,2,3],
              'B': [4,5,6],
              'C': [7,8,9],
              'D': [1,2,3],
              'E': [4,5,6],
              'F': [7,8,9]}
subproblem_list = split_generic(input_dict,2, verbose=True)
check_dims(subproblem_list, input_dict)


topo_fname = "../topologies/topology-zoo/Ion.graphml"#GtsCe.graphml"
#tm_fname = "../traffic-matrices/uniform/GtsCe.graphml_uniform_1475504323_64.0_0.05_traffic-matrix.pkl"
tm_fname = "../traffic-matrices/uniform/Ion.graphml_uniform_1545787193_64.0_0.15_traffic-matrix.pkl"
num_paths, edge_disjoint, dist_metric = PATH_FORM_HYPERPARAMS
    
pf_original = PathFormulation.new_max_flow(
    num_paths,
    edge_disjoint=edge_disjoint,
    dist_metric=dist_metric)
with open('path-form.csv', 'a') as results:
    print_(','.join(HEADERS), file=results)
    
    problem = Problem.from_file(topo_fname, tm_fname)

    paths_dict = pf_original.get_paths(problem)
    com_list = problem.commodity_list
    num_edges = len(problem.G.edges)
    enum_edges_dict = {}
    for i, edge in enumerate(problem.G.edges):
        enum_edges_dict[edge] = i

    # create dictionary of all edges used by each commodity
    com_path_edges_dict = defaultdict(list)
    min_demand = np.inf
    max_demand = 0
    for k, (source, target, demand) in com_list:
        paths_array = paths_dict[(source, target)]
        if min_demand > demand:
            min_demand = demand
        if max_demand < demand:
            max_demand = demand
            
        for path in paths_array:
            com_path_edges_dict[(k, source, target, demand)] += list(path_to_edge_list(path))

    com_path_edges_onehot_dict = defaultdict(list)
    for (k, source, target, demand), edge_list in com_path_edges_dict.items():
        onehot_edge = [0]*num_edges
        for edge in edge_list:
            edge_i = enum_edges_dict[edge]
            onehot_edge[edge_i] = 1
        # add in normalized demand as a dimension
        com_path_edges_onehot_dict[k] = onehot_edge + [(demand - min_demand)/(max_demand-min_demand)]
        
    print("creating subproblem list...")
    subproblem_list_meansplit = split_generic(com_path_edges_onehot_dict, 2, verbose=False)
    

Assigned 0 entities
[[[], [], []], [[1], [2], [3]]]
[[], ['A']]


[[[4], [5], [6]], [[1], [2], [3]]]
[['B'], ['A']]


[[[4], [5], [6]], [[1, 7], [2, 8], [3, 9]]]
[['B'], ['A', 'C']]


[[[4, 1], [5, 2], [6, 3]], [[1, 7], [2, 8], [3, 9]]]
[['B', 'D'], ['A', 'C']]


[[[4, 1, 4], [5, 2, 5], [6, 3, 6]], [[1, 7], [2, 8], [3, 9]]]
[['B', 'D', 'E'], ['A', 'C']]


[[[4, 1, 4], [5, 2, 5], [6, 3, 6]], [[1, 7, 7], [2, 8, 8], [3, 9, 9]]]
[['B', 'D', 'E'], ['A', 'C', 'F']]


checking split of 3 dimensions over 2 subproblems
subproblem 0: [3. 4. 5.]
subproblem 1: [5. 6. 7.]
Loading paths from pickle file /lfs/1/fiodar/ncflow/topologies/paths/path-form/Ion.graphml-4-paths_edge-disjoint-True_dist-metric-inv-cap-dict.pkl
paths_dict size: 15500
creating subproblem list...
Assigned 0 entities
Assigned 1000 entities
Assigned 2000 entities
Assigned 3000 entities
Assigned 4000 entities
Assigned 5000 entities
Assigned 6000 entities
Assigned 7000 entities
Assigned 8000 entities
Assigned 9000 entities
Assigned 

In [31]:
print("checking dims...")
subproblem_dim_means = check_dims(subproblem_list_meansplit, com_path_edges_onehot_dict)
print(subproblem_dim_means[0] - subproblem_dim_means[1])
print(np.mean(abs(subproblem_dim_means[0] - subproblem_dim_means[1])))

checking dims...
checking split of 293 dimensions over 2 subproblems
subproblem 0: [0.13058065 0.13225806 0.064      0.06090323 0.07122581 0.07109677
 0.07058065 0.00477419 0.05303226 0.00464516 0.01122581 0.008
 0.04980645 0.04916129 0.008      0.008      0.21045161 0.21070968
 0.13109677 0.12903226 0.18322581 0.18825806 0.18464516 0.18980645
 0.008      0.19354839 0.20206452 0.02516129 0.02374194 0.03096774
 0.03058065 0.02503226 0.02374194 0.0196129  0.02129032 0.18309677
 0.20645161 0.12696774 0.004      0.19793548 0.20012903 0.04941935
 0.05535484 0.08451613 0.04374194 0.12180645 0.0083871  0.00154839
 0.0076129  0.10877419 0.108      0.05419355 0.0476129  0.13251613
 0.12851613 0.11096774 0.20593548 0.14709677 0.11135484 0.10503226
 0.13496774 0.11380645 0.07587097 0.01922581 0.06929032 0.00787097
 0.07574194 0.09419355 0.02012903 0.02283871 0.02064516 0.02129032
 0.01896774 0.02245161 0.02090323 0.06541935 0.06670968 0.06812903
 0.00980645 0.06       0.044      0.0516129  0.0452

# Firas's Work

In [ ]:
problem_name, topo_fname, tm_fname = p8
prob = Problem.from_file(topo_fname, tm_fname)
NUM_PATHS = 4
NUM_SUBPROBLEMS = 8

In [ ]:
def solve_subproblems(problem_list):
    sol_dicts, runtimes, obj_vals = [], [], []
    for sub_problem in problem_list:
        pf = PathFormulation.get_pf_for_obj(Objective.MAX_FLOW, NUM_PATHS)
        pf.solve(sub_problem)
        sol_dicts.append(pf.extract_sol_as_dict())
        runtimes.append(pf.runtime)
        obj_vals.append(pf.obj_val)
    #check
    return sol_dicts, runtimes, obj_vals

def solve_and_check_feasiblity(problem, num_subproblems, num_paths):    
    paths_dict = PathFormulation.new_max_flow(num_paths).get_paths(problem)
    problem_list = split_problem_smartpath(prob, num_subproblems, paths_dict)
    sol_dicts, runtimes, obj_vals = solve_subproblems(problem_list)
    check_feasibility(problem, sol_dicts)

In [ ]:
solve_and_check_feasiblity(prob, NUM_SUBPROBLEMS, NUM_PATHS)

# End Firas's Work

In [ ]:
def validate_solution(sol_dicts_all, num_subproblems):
    for obj_type in obj_types:
        sol_dicts = sol_dicts_all[obj_type]
        for p_spec in problems:
            problem = Problem.from_file(p_spec[1], p_spec[2a])
            com_list = problem.commodity_list
            
            for n_i, n in enumerate(num_subproblems):
                merged_sol_dict = defaultdict(int)
                for j in range(n):
                    sol_dict = sol_dicts[p_spec][n_i][j]
                    total_flow = 0
                    for commod_key, flow_list in sol_dict.items():
                        assert_flow_conservation(flow_list, commod_key)
                        src, target = commod_key[-1][0], commod_key[-1][1]

                        flow = compute_in_or_out_flow(flow_list, 0, {commod_key[-1][0]})

                        merged_sol_dict[(src,target)] += flow
            
                frac_demands_satisfied = {commod_key:
                                      merged_sol_dict[(commod_key[-1][0],
                                                       commod_key[-1][1])] / commod_key[-1][-1]
                                      for commod_key in com_list}
                for commod_key, frac in frac_demands_satisfied.items():
                    if frac > 1:
                        print("assertion error, demand oversatisfied "+ str(commod_key) + " " + str(frac))
                        break
                    

In [ ]:
results_all_obj = {}
runtimes_all_obj = {}
sol_dicts_all_obj = {}
for obj_type in obj_types:
    
    results, runtimes, sol_dicts = benchmark_split(problems, num_subproblems, obj_type, smart=False)
    results_all_obj[obj_type] = results
    runtimes_all_obj[obj_type] = runtimes
    sol_dicts_all_obj[obj_type] = sol_dicts

In [ ]:
results_all_obj_smart = {}
runtimes_all_obj_smart = {}
sol_dicts_all_obj_smart = {}
for obj_type in obj_types:
    
    results, runtimes, sol_dicts = benchmark_split(problems, num_subproblems, obj_type, smart=True, generic=False)
    results_all_obj_smart[obj_type] = results
    runtimes_all_obj_smart[obj_type] = runtimes
    sol_dicts_all_obj_smart[obj_type] = sol_dicts

In [ ]:
results_all_obj_smart_gen = {}
runtimes_all_obj_smart_gen = {}
sol_dicts_all_obj_smart_gen = {}
for obj_type in obj_types:
    
    results, runtimes, sol_dicts = benchmark_split(problems, num_subproblems, obj_type, smart=True, generic=True)
    results_all_obj_smart_gen[obj_type] = results
    runtimes_all_obj_smart_gen[obj_type] = runtimes
    sol_dicts_all_obj_smart_gen[obj_type] = sol_dicts

In [ ]:
smart_results = [results_all_obj_smart, runtimes_all_obj_smart, sol_dicts_all_obj_smart]
pickle.dump(smart_results, open("results/smart_results.pkl", "wb"))

In [ ]:
[results_all_obj_smart, 
runtimes_all_obj_smart, 
sol_dicts_all_obj_smart] = pickle.load(open("results/smart_results.pkl", "rb"))

In [ ]:
#run NCFlow on problem set
problems_ncflow = [(os.path.basename(p[1]), p[1], p[2]) for p in problems]
results_ncflow, runtimes_ncflow = ncflow.benchmark(problems_ncflow)

In [ ]:
colors = ['k', 'r', 'b', 'g']
linestyles = ['-', '--', ':']
#print(results_ncflow)
#results_ncflow = results_ncflow[1::2]
#print(runtimes_ncflow)
def compute_obj_val(obj, problem, obj_vals, sol_dicts):
    if obj == 'max_flow':
        return sum(obj_vals)
    elif obj == 'min_max_link_util':
        link_utils_dict = defaultdict(int)
        for sol_dict in sol_dicts:
            for flow_list in sol_dict.values():
                for ((u, v), flow_val) in flow_list: # TODO: is this right? Or is it ((u, v), flow_val)?
                    link_utils_dict[(u, v)] += flow_val
        
        # TODO: is this right? Or is it ((u, v), c_e)?
        link_utils = {(u, v): link_utils_dict[(u, v)] / c_e for (u, v, c_e) in problem.G.edges.data('capacity')}
        return max(link_utils.values())
    
# run the benchmarks, plot results
def plot_benchmark(results_all, runtimes_all, sol_dicts_all, results_ncflow, runtimes_ncflow,
                  results_cspf, runtimes_cspf):
    for obj_type in obj_types:
        
        results = results_all[obj_type]
        runtimes = runtimes_all[obj_type]
        sol_dicts = sol_dicts_all[obj_type]
        
        fig, ax = plt.subplots()
        for p_i, p in enumerate(problems):
            total_obj_values = []
            for n_i, n in enumerate(num_subproblems):
                total_obj_val = compute_obj_val(obj_type, Problem.from_file(p[1], p[2]),
                                                results[p][n_i], sol_dicts[p][n_i])
        #         sum_val = sum(results[p][n_i])
                total_obj_values.append(total_obj_val)
            ax.plot(num_subproblems, total_obj_values, marker='*', c=colors[p_i%len(colors)], linestyle=linestyles[p_i%len(linestyles)],
                    label=p[0])
            #if (obj_type == "max_flow"):
                #also plot flow achieved by ncflow
                #ax.plot(1, results_ncflow[p_i], marker='P', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; ncflow")
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlabel('number of subproblems')
        plt.ylabel('total flow')
        #plt.ylim([0,None])
        #plt.savefig('./plots/flow_'+p[0]+'.png', bbox_inches='tight')

        fig, ax = plt.subplots()
        for p_i, p in enumerate(problems):
            total_runtime = []
            max_runtime = []
            for n_i, n in enumerate(num_subproblems):
                sum_val = sum(runtimes[p][n_i])
                print(sum_val)
                max_time = max(runtimes[p][n_i])
                total_runtime.append(sum_val)
                max_runtime.append(max_time)
            ax.plot(num_subproblems, total_runtime, marker='*', linestyle=linestyles[0], 
                    c=colors[p_i%len(colors)], label='sum ' + p[0])
            ax.plot(num_subproblems, max_runtime, marker='^', linestyle=linestyles[1], 
                    c=colors[p_i%len(colors)], label='max ' + p[0])
            if (obj_type == "max_flow"):
                #also plot runtime of ncflow
                ax.plot(1, runtimes_ncflow[p_i], marker='P', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; ncflow")
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlabel('number of subproblems')
        plt.ylabel('execution time (seconds)')
        plt.ylim([0,None])
        #plt.savefig('./plots/runtime_'+p[0]+'.png', bbox_inches='tight')

        # plot performance vs runtime.
        fig, ax = plt.subplots()
        for p_i, p in enumerate(problems):
            total_obj_values = []
            total_runtime = []
            max_runtime = []
            for n_i, n in enumerate(num_subproblems):
                total_obj_val = compute_obj_val(obj_type, Problem.from_file(p[1], p[2]),
                                                results[p][n_i], sol_dicts[p][n_i])
                total_obj_values.append(total_obj_val)

                sum_val = sum(runtimes[p][n_i])
                max_time = max(runtimes[p][n_i])
                total_runtime.append(sum_val)
                max_runtime.append(max_time)
            print(max_runtime)
            print(total_obj_values)
            ax.scatter(max_runtime, total_obj_values, marker='*', c=colors[p_i%len(colors)], label=p[0])
            for n_i, n in enumerate(num_subproblems):
                ax.annotate(n, (max_runtime[n_i], total_obj_values[n_i]))
            if (obj_type == "max_flow"):
                #also plot runtime of ncflow
                ax.plot(runtimes_ncflow[p_i], results_ncflow[p_i], marker='P', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; ncflow")
                ax.plot(runtimes_cspf[p_i], results_cspf[p_i], marker='x', markersize=10, c=colors[p_i%len(colors)], label=p[0]+"; CSPF")
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlabel('execution time (seconds)')
        plt.ylabel('total allocated flow')
        ymin, ymax = plt.ylim()
        plt.ylim([0,ymax*1.1])
        ax.set_xscale('log')
        """
        for p_spec in probs:
            break
            problem = Problem.from_file(p_spec[1], p_spec[2])
            com_list = problem.commodity_list

            fig,ax = plt.subplots()

            # plot the demand distribution
            demands = [ com_list[i][1][2] for i in range(len(com_list)) ]
            num_bins = 100
            counts, bin_edges = np.histogram(demands, bins=num_bins)
            cdf = np.cumsum(counts)
            ax.plot(bin_edges[1:], cdf/cdf[-1], c='m', label="demands")

            for n_i, n in enumerate(num_subproblems):
                merged_sol_dict = defaultdict(int)
                for j in range(n):
                    sol_dict = sol_dicts[p_spec][n_i][j]


                    total_flow = 0
                    for commod_key, flow_list in sol_dict.items():
                        src, target = commod_key[-1][0], commod_key[-1][1]

                        flow = compute_in_or_out_flow(flow_list, 0, {commod_key[-1][0]})

                        merged_sol_dict[(src,target)] += flow

                    # get amount of flow assigned to each commodity (ASSUMING SINGLE PATH)
                    #flow_counts += [ sol_dict[sdf][0][1] for sdf in sol_dict if len(sol_dict[sdf]) > 0 ]

                frac_demands_satisfied = {commod_key:
                                          merged_sol_dict[(commod_key[-1][0],
                                                           commod_key[-1][1])] / commod_key[-1][-1]
                                          for commod_key in com_list}

                # take frac_demands_satisfied, extract list, plot cdf
                num_bins = 100
                #print("sol_dict: " + str(sum(flow_counts)) + ", actual obj: " + str(sum(results[p][n_i])))
                counts, bin_edges = np.histogram(flow_counts, bins=num_bins)
                cdf = np.cumsum(counts)
                ax.plot(bin_edges[1:], cdf/cdf[-1], c=colors[n_i], label=str(n)+" subproblems")

            plt.xlabel('per-commodity allocated flow')
            plt.ylabel('cumulative')
            plt.legend()
"""
    

In [ ]:
#problems = [p6]
#plot_benchmark(results_all_obj, runtimes_all_obj, sol_dicts_all_obj, 
#               results_ncflow, runtimes_ncflow)
#results_ncflow = None
#runtimes_ncflow = None
plot_benchmark(results_all_obj_smart, runtimes_all_obj_smart, 
               sol_dicts_all_obj_smart, results_ncflow, runtimes_ncflow,
               results_cspf, runtimes_cspf)

plot_benchmark(results_all_obj_smart_gen, runtimes_all_obj_smart_gen, 
               sol_dicts_all_obj_smart_gen, results_ncflow, runtimes_ncflow,
               results_cspf, runtimes_cspf)


In [ ]:
num_paths, edge_disjoint, dist_metric = PATH_FORM_HYPERPARAMS
#problem = Problem.from_file("../topologies/topology-zoo/GtsCe.graphml", 
#                            "../traffic-matrices/uniform/GtsCe.graphml_uniform_1475504323_64.0_0.05_traffic-matrix.pkl")
#problem.G
#print(problem.G.edges.data())
#com_list = problem.commodity_list
#problem2 = problem.copy()
#print(dir(problem))

#pf = PathFormulation.new_max_flow(
#                    num_paths,
#                    edge_disjoint=edge_disjoint,
#                    dist_metric=dist_metric)
        
#paths_dict = pf.get_paths(problem)

#com0 = com_list[3]
#print(com0)
#print(paths_dict[(com0[1][0],com0[1][1])])

"""
problem2.traffic_matrix.tm
new_tm = problem2.traffic_matrix.tm[0:10,:]

num_rows = len(problem2.traffic_matrix.tm)

shuffled_indices = list(range(num_rows))
random.shuffle(shuffled_indices)

num_first_problem = math.floor(num_rows/2)

for i in shuffled_indices[1:num_first_problem]:
    problem2.traffic_matrix.tm[i,:] = 0

#print(problem2.traffic_matrix.tm[1:5,:])

for u,v in problem.G.edges:
    problem.G[u][v]['capacity'] = problem.G[u][v]['capacity']/2
    problem2.G[u][v]['capacity'] = problem2.G[u][v]['capacity']/2
"""

In [ ]:
def CSPF(problems):
    
    results_all = []
    runtimes_all = []
    
    for problem_name, topo_fname, tm_fname in problems:
        problem = Problem.from_file(topo_fname, tm_fname)
        
        com_list = problem.commodity_list
        tm = problem.traffic_matrix.tm
        
        pf = PathFormulation.new_max_flow(
                    num_paths,
                    edge_disjoint=edge_disjoint,
                    dist_metric=dist_metric)
        
        paths_dict = pf.get_paths(problem)
        
        # initialize link capacity dict
        remaining_link_capacity_dict = {}
        for u,v in problem.G.edges:
            remaining_link_capacity_dict[(u,v)] = problem.G[u][v]['capacity']
            
        # sort paths in ascending order
        all_paths_list = []
        allocated_coms = {}
        for k, (source, target, demand) in com_list:
            paths_array = paths_dict[(source, target)]
            all_paths_list += paths_array
            allocated_coms[(source,target)] = False
        all_paths_list.sort(key=len)
        
        # iterate through sorted paths
        total_allocated_flow = 0
        startTime = datetime.now()
        for path in all_paths_list:
            source = path[0]
            target = path[-1]
            demand = tm[source,target]
            
            # skip if we have already allocated this commodity
            if allocated_coms[(source,target)]:
                continue
            
            # check that each edge in list has enough capacity
            edge_list = list(path_to_edge_list(path))
            room = True
            for u,v in edge_list:
                if remaining_link_capacity_dict[(u,v)] < demand:
                    room = False
                    break
            
            if not room:
                continue
            
            # allocate
            for u,v in edge_list:
                remaining_link_capacity_dict[(u,v)] -= demand
            allocated_coms[(source, target)] = True
            total_allocated_flow += demand
        runtime = datetime.now() - startTime
        
        results_all.append(total_allocated_flow)
        runtimes_all.append(runtime.total_seconds())
        print("Runtime: " + str(runtime))
        print("Allocated Flow:" + str(total_allocated_flow))
    return results_all, runtimes_all

In [ ]:
results_cspf, runtimes_cspf = CSPF(problems)

In [ ]:
print(results_ncflow)